# TensorFlow Tutorial

뉴럴넷과 CNN에 대해 기본적인 강의를 듣고 기초지식을 가지고 있다는 가정하에 진행되는 튜토리얼입니다.

In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = '-1' # for using only cpu
import tensorflow as tf
import numpy as np

C:\Users\yangyangii\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Basic Code

In [2]:
a = tf.constant([2],dtype=tf.float32)
b = tf.constant([3],dtype=tf.float32)
c = tf.constant([4],dtype=tf.float32)

op = a*b+c
# op = tf.add(tf.multiply(a,b), c)

sess = tf.Session()
result = sess.run(op)
print(result)
sess.close()

[10.]


tf.constant는 텐서플로우에서 상수값으로 사용하기 위한 타입입니다.
코드를 보면 각각 a, b, c가 2, 3, 4라는 값으로 데이터 타입은 float32로 할당되었습니다.
이 상수값을 가진 텐서를 통해 a*b+c라는 operation을 구현해서 op라는 이름으로 저장합니다.

여기서는 간단히 연산 기호를 통해 구현했지만, 실제로는 주석에 있는 tf.add와 tf.multiply라는 operation 함수로 매칭됩니다.
또한 텐서플로우는 Session을 통해 그래프를 작동시키는데, 이를 위해 필요한 가장 간단한 함수들을 볼 숫 있습니다.

tf.constant

In [3]:
# Constant 1-D Tensor populated with value list.
tensor1 = tf.constant([1, 2, 3, 4, 5, 6, 7]) # [1 2 3 4 5 6 7]

# Constant 2-D tensor populated with scalar value -1.
tensor2 = tf.constant(-1.0, shape=[2, 3]) # [[-1. -1. -1.], [-1. -1. -1.]]

sess = tf.Session()
print(sess.run(tensor1))
print(sess.run(tensor2))
print(tensor1)
print(tensor2)
sess.close()

[1 2 3 4 5 6 7]
[[-1. -1. -1.]
 [-1. -1. -1.]]
Tensor("Const_3:0", shape=(7,), dtype=int32)
Tensor("Const_4:0", shape=(2, 3), dtype=float32)


이 코드는 tf.constant를 사용하되 Rank와 Shape의 크기를 늘린 예시입니다.
단순히 텐서를 실행했을 때의 결과물을 볼 수 있습니다.

뉴럴넷은 대부분 행렬 연산을 통해 이루어지기 때문에, 이러한 2차원 3차원들에 대해 익숙해 지시면 용이할 것 같습니다.

텐서 자체를 프린트 했을 때의 내용을 보면, 각 텐서의 shape과 data dtype을 확인할 수 있습니다. 뉴럴넷에서 연산을 프로그래밍 할 때는 이 shape을 계속해서 주의하면서 모델을 구축해줘야 합니다.

tf.placeholder

In [4]:
input_data = [1,2,3,4,5]

x = tf.placeholder(dtype=tf.float32)
k = tf.constant(2, dtype=tf.float32)
y = x * k

sess = tf.Session()
result = sess.run(y, feed_dict={x:input_data})

print(result)

[ 2.  4.  6.  8. 10.]


이번에는 tf.placeholder를 통해 feed를 하는 예시입니다.
먼저 x라는 placeholder 텐서를 만들고 operation을 만든 다음에 최종적으로 세션을 실행 할 때 feed_dict에서 placeholder에 어떤 데이터를 넣을지 정해주는 코드입니다.

x의 shape이 정해져있지 않으므로 스칼라로 취급되며, 각각의 input data 스칼라에 constant 2를 곱하게 됩니다.

In [47]:
input_data = [1,2,3,4,5]

x = tf.placeholder(dtype=tf.float32)
W = tf.Variable([10],dtype=tf.float32)
W = W.assign_add([10.0])
y = W*x

sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)

result = sess.run(y, feed_dict={x:input_data})

print(result)

[  20.   40.   60.   80.  100.]


tf.Variable은 말 그대로 변수로 사용하기 위한 텐서입니다.
주로 학습에 사용되는 parameter를 만드는데 사용됩니다.

tf.Variable은 사용하기에 앞서 반드시 초기화가 필요합니다

위 코드는 tf.Variable을 사용하기 위해 초기화를 추가한 것과, tf.Variable을 통해 만든 W가 값이 변할수 있다는 것을 보여준 코드입니다.

# MNIST w/ Single Layer Perceptron

MNIST Data

본격적으로 뉴럴넷을 구현하기 위한 모델 구축에 들어갑니다.

In [6]:
import tensorflow as tf
import numpy as np
import scipy.misc

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST/', one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
mnist.train.images.shape

(55000, 784)

In [9]:
scipy.misc.toimage(mnist.train.images[999].reshape((28,28)))

In [11]:
mnist.train.labels[999]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

위 코드들은 우리가 뉴럴넷에 사용할 MNIST 데이터를 가져오고, 데이터가 정상적으로 들어가 있는지 확인해보는 코드입니다.

MNIST 데이터는 손글씨 digit 데이터로 (28,28)의 shape을 갖춘 gray scale 이미지 입니다.
여기서 텐서플로우에 내장된 mnist 데이터를 가져오는 함수를 통해 가져오게 되면, 다운받아져 있지 않은 경우 다운로드를 하게 되며, 함수의 첫번째 parameter에 해당하는 경로에 다운을 받습니다.

one_hot의 경우 classifier에 쓰기 위해 label의 종류만큼 size를 정하고 label에 해당하는 곳에만 sparse하게 1을 부여하는 encoding을 하게 되는데, digit의 경우 0-9까지 10가지가 있으므로 shape이 [10]이라고 생각하시면 되겠습니다. 예시를 보시면 쉽게 이해할 수 있습니다.

In [12]:
images = tf.placeholder(shape=[None, 784], dtype=tf.float32)
labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)

W = tf.Variable(tf.truncated_normal([784, 10], stddev=0.1))
b = tf.Variable(tf.zeros([10]))

images와 labels는 우리가 MNIST 데이터를 순차적으로 집어넣을 것이므로 tf.placeholder를 통해 만드는데, 학습할 때 MNIST 데이터를 하나씩만 하는 것이 아니라 mini-batch size만큼씩 병렬처리할 것이므로 mini-batch size만큼 shape을 정해주면 되는데, 텐서플로우는 placeholder에서 python의 None 타입을 통해서 추후에 mini-batch size만큼 자동할당 되도록 돕고 있습니다. 따라서 우리는 여기서는 None으로 해놓고 뒤에서 mini-batch size를 정할 겁니다.

W와 b는 Weight와 bias입니다. 뉴럴넷에서 배운 학습 parameter인데 우리는 간단한 singe layer perceptron을 위해 input image의 size인 784와 output의 size인 10에 대해 weight와 bias의 shape을 정하고 각각 텐서를 만듭니다.

tf.truncated_normal은 랜덤함수를 통해 표준편차 0.1에서 아웃라이어를 잘라낸 상태의 값으로 shape에 맞춰져 있는 텐서를 리턴합니다.

tf.zeros는 shape에 맞춰 모든 값을 0으로 되어있는 텐서를 리턴합니다.

In [13]:
logit = tf.nn.softmax(tf.matmul(images, W) + b)
loss = -tf.reduce_sum(labels*tf.log(logit))

tf.matmul은 행렬곱 함수이며, image와 W를 행렬곱 하고 bias를 더한 다음에 softmax를 취하는 operation을 logit에 저장합니다.

loss는 cross entropy 식으로 구현했고, tf.reduce_sum은 axis에 따라 텐서의 모든 값을 더해서 차원을 줄이고 리턴하는 함수입니다.

In [14]:
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

학습에 사용되는 optimizer로는 기본적인 Gradient Descent를 사용하고, learning rate는 0.01로 설정합니다.

또한, cross entropy loss를 minimize하는 방식으로 최적화해야하기 때문에 minimize 함수를 사용합니다.

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [16]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_op, feed_dict={images: batch_xs, labels: batch_ys})

1천번 학습을 시킬텐데 1번의 iteration마다 mini-batch size가 100이므로 100개의 example을 보고나서 최적화 update를 한 번 합니다.

batch_xs와 batch_ys에는 각각 100개의 이미지([100, 784])와 Label([100, 10])이 들어가 있고, iteration마다 다음 mini-batch의 값을 꺼내서 placeholder에 feed 하도록 합니다.

Accuracy

In [17]:
correct_prediction = tf.equal(tf.argmax(logit, axis=1), tf.argmax(labels, axis=1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

다음은 모델의 학습을 통해 성능이 얼마나 좋아졌는지 확인하는 단계입니다.

성능을 판단하기 위한 함수를 만들기 위해 모델의 아웃풋과 label이 얼마나 일치하는지 확인합니다.

tf.argmax는 axis를 기준으로 각각 가장 큰 값의 index를 리턴하는 함수입니다.

tf.equal은 두 Tensor의 element들이 일치하는지에 대해 Bool type을 리턴하는 함수입니다.

tf.reduce_mean은 axis를 기준으로 평균을 구하는 함수이며, 평균을 위해서 float 타입만 취급하기 때문에 이전에 tf.cast를 통해 타입을 float로 변경해 줍니다.

In [18]:
print(sess.run(acc, feed_dict={images: mnist.train.images, labels: mnist.train.labels}))

0.89910907


In [19]:
print(sess.run(acc, feed_dict={images: mnist.test.images, labels: mnist.test.labels}))

0.9032


각각 Training Set Accuracy, Test Set Accuracy 입니다.

# Practice MLP

In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [25]:
images = tf.placeholder(shape=[None, 784], dtype=tf.float32)
labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)
# W = tf.Variable(tf.zeros([784, 10]))
W1 = tf.Variable(tf.truncated_normal([784, 100], stddev=0.1))
b1 = tf.Variable(tf.zeros([100]))
W2 = tf.Variable(tf.truncated_normal([100, 10], stddev=0.1))
b2 = tf.Variable(tf.zeros([10]))

In [49]:
out1 = tf.nn.relu(tf.matmul(images, W1) + b1)
logit = tf.matmul(out1, W2) + b2
# loss = -tf.reduce_sum(labels*tf.log(logit))
# loss = tf.nn.softmax_cross_entropy_with_logits()
loss = tf.reduce_sum(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logit)
)

In [50]:
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [51]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [112]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100, shuffle=True)
    _, batch_loss = sess.run([train_op, loss], feed_dict={images: batch_xs, labels: batch_ys})
print(batch_loss)

146.492


loss

In [113]:
acc = tf.reduce_mean(tf.cast(
    tf.equal(tf.argmax(logit, axis=1), tf.argmax(labels, axis=1)),
    tf.float32))

In [114]:
print(sess.run(acc, feed_dict={images: mnist.train.images, labels: mnist.train.labels}))

0.991273


In [115]:
print(sess.run(acc, feed_dict={images: mnist.test.images, labels: mnist.test.labels}))

0.9769


실습! 성능 높이기: 15분

# MNIST w/ CNN

In [20]:
import tensorflow as tf
import numpy as np
import scipy.misc

In [21]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST/', one_hot=True)

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [22]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

CNN 부터는 모델이 조금씩 복잡해지므로 단순화 시킬 수 있는 부분들에 대해서는 함수나 클래스로 reusable하게 만드는 것이 중요합니다.

weight와 bias를 만드는 부분은 모든 레이어에 대해 필요한 부분이며, 대부분 단순 반복하고 shape만 달라지므로 shape을 parameter로 기본적인 함수를 정의합니다. MLP에서 구현했던 것과 거의 동일합니다.

0.1로 bias를 초기화하는 이유는 non-linear activation function으로 ReLU를 사용할 경우 죽는 Neuron이 발생하는 것을 방지하기 위해서입니다.

In [23]:
def conv(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

tf.nn.conv2d와 tf.nn.max_pool은 Convolutional layer와 Maxpooling layer를 구현하는 함수입니다.

tf.nn.conv2d는 2-dim input을 기준으로 input x가 들어왔을 때 W를 기준으로 kernel을 구성하여 multiple convolution을 한 아웃풋을 리턴합니다. padding은 zero padding을 설정하는 parameter이고 'SAME'일 경우 shape을 유지시키고 'VALID'의 경우 padding없이 유효한 값에 대해서만 연산합니다. strides는 리스트의 첫번째와 마지막은 batch와 depth를 나타내므로 convoltion을 할 때는 무조건 1로 설정한다고 생각하시면 되고, 2번째와 3번째는 각각 height와 width의 stride를 설정해주시면 됩니다. 여기서 우리는 간단한 CNN을 구현할 것이기 때문에 stride는 1로 설정하고 feature map의 size는 max pooling layer를 통해 줄이도록 합니다.

tf.nn.max_pool은 input x에 대해 max-pooling을 적용하며, ksize는 커널의 사이즈를 의미합니다. 마찬가지로 2번째와 3번째가 height와 width이므로, height와 width를 2씩, stride 또한 2씩 설정합니다

In [24]:
images = tf.placeholder(shape=[None, 784], dtype=tf.float32)
reshape_images = tf.reshape(images, [-1, 28, 28, 1])

labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)

기본적으로 input과 label을 placeholder를 통해 받아들이는 부분은 동일하나, Convolution 연산을 위해서는 2차원이어야 하고 텐서플로우의 conv2d 함수는 4차원 텐서를 인풋으로 받기 때문에, tf.reshape을 통해 dimension을 조정해줍니다.

conv2d는 4-D Tensor를 input으로 받는데 리스트의 첫번째인 -1은 mini-batch size에 따라 자동적으로 설정되게 하는 부분이며, 마지막의 1은 gray scale이므로 depth 1을 넣습니다. RGB image의 경우 3이 들어가게 됩니다.

In [25]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

Weight와 bias는 설계한 모델에 따라 shape을 맞춰서 만들어주면 됩니다.

Weight는 4 rank의 shape으로 구성되는데, 앞의 둘은 kernel의 height와 width라고 생각하면 되고, input channel, output channel의 size라고 생각하시면 됩니다.

bias는 output channel의 size입니다.

max-pooling의 경우 weight가 필요하지 않으므로 따로 설정하지 않고, Fully-connected layer에 대해 weight와 bias를 설정하면 됩니다. 이는 MLP에서 한 부분과 동일합니다.

In [26]:
conv_layer1 = tf.nn.relu(conv(reshape_images, W_conv1) + b_conv1)
pool_layer1 = max_pool(conv_layer1)

conv_layer2 = tf.nn.relu(conv(pool_layer1, W_conv2) + b_conv2)
pool_layer2 = max_pool(conv_layer2)

pool2_flat = tf.reshape(pool_layer2, [-1, 7*7*64])
fc_layer1 = tf.nn.relu(tf.matmul(pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
drop_out = tf.nn.dropout(fc_layer1, keep_prob)

logit = tf.matmul(drop_out, W_fc2) + b_fc2

위에서 구현한 각 함수들과 학습 parameter들을 통해 레이어들을 구성하는 operation들입니다.
convolutional layer에서는 ReLU 함수를 activation 함수로 사용했습니다.

주의할 점은 Conv는 4-D Tensor가 input이지만 Fully-connected layer의 input은 2-D Tensor이므로 맞춰줘야한다.

두번의 max-pooling layer를 거치게 되면 기존의 이미지가 [28,28]이고 stride가 2 이므로 [28,28] -> [14,14] -> [7,7]이 된다.

[7,7]의 feature map이 마지막 channel의 개수 64개만큼 있으므로 총 7*7*64개의 노드가 있다.

-1은 mini-batch size만큼 해당 노드 수로 쫙 펼쳐준 것이다.

Convolution -> ReLU -> Max-pooling
-> Convolution -> ReLU -> Max-pooling
-> Linear -> ReLU -> Dropout
-> Linear -> Softmax

위와 같은 순서로 진행되며, Softmax는 사실 위에 포함되어 있지 않고 아래 코드에서 진행된다.

In [27]:
loss = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logit)
    tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=labels)
)
train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(logit, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

BATCH_SIZE = 50
max_epoch = 1
total_batch = int(mnist.train.num_examples/BATCH_SIZE)

# with tf.Session() as sess:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(max_epoch):
    for i in range(total_batch):
        batch = mnist.train.next_batch(BATCH_SIZE, shuffle=True)
        sess.run(train_op, 
             feed_dict={images: batch[0], labels: batch[1], keep_prob: 0.5}
        )
        train_acc = sess.run(accuracy,
            feed_dict={images: batch[0], labels: batch[1], keep_prob: 1.0}
        )
        print('step %d, training accuracy %g' % (i, train_acc))
        if i % 100 == 0:
            test_acc = sess.run(accuracy, 
                 feed_dict={images: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}
            )
            print('test accuracy %g' % test_acc)
test_acc = sess.run(accuracy, 
     feed_dict={images: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}
)
print('test accuracy %g' % test_acc)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

step 0, training accuracy 0.08
test accuracy 0.068
step 1, training accuracy 0.16
step 2, training accuracy 0.08
step 3, training accuracy 0.14
step 4, training accuracy 0.16
step 5, training accuracy 0.22
step 6, training accuracy 0.14
step 7, training accuracy 0.1
step 8, training accuracy 0.18
step 9, training accuracy 0.22
step 10, training accuracy 0.18
step 11, training accuracy 0.18
step 12, training accuracy 0.18
step 13, training accuracy 0.2
step 14, training accuracy 0.24
step 15, training accuracy 0.3
step 16, training accuracy 0.28
step 17, training accuracy 0.3
step 18, training accuracy 0.36
step 19, training accuracy 0.36
step 20, training accuracy 0.34
step 21, training accuracy 0.3
step 22, training accuracy 0.52
step 23, training accuracy 0.46
step 24, training accuracy 0.3
step 25

KeyboardInterrupt: 

loss를 softmax와 cross entropy를 함께 처리해주는 함수를 통해 처리한다.

학습 optimizer 함수는 여러가지가 있으나 AdamOptimizer가 잘 작동한다고 알려져 있으므로 이를 사용하고 learning rate는 적당히 작은 값인 0.0001을 사용한다.

mini-batch size는 50으로 하고, 총 iteration은 500번으로 한다.

drop out의 비율을 학습할 때는 0.5로 하여 절반을 drop시키고 실제 예측할 때는 1.0으로 하여 모두 켜두도록 한다.

학습시 마다 training set의 accuracy를 확인하고, 100번째마다 test set의 accuracy를 확인한다.

실습! 성능 높이기: 15분, epoch 1000 이내

# Prediction w/ CNN

실습! Prediction 해보기: 10분

In [31]:
input_data = mnist.train.images[999:1000]
print(np.argmax(mnist.train.labels[999]))
scipy.misc.toimage(input_data[0].reshape((28,28)))

2


In [30]:
pred = tf.argmax(logit, 1)
result = sess.run(pred, feed_dict={images: input_data, keep_prob: 1.0})
print('Prediction Result: {}'.format(result[0]))

Prediction Result: 2


숫자 2에 대한 이미지를 예측하고 결과를 보여주는 함수 구현

# Tensorboard w/ CNN

In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def weight_variable(shape, name='weights'):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name='biases'):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

In [4]:
def conv(x, W, name='conv'):
    with tf.name_scope(name) as scope:
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x, name='max_pool'):
    with tf.name_scope(name) as scope:
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [5]:
images = tf.placeholder(shape=[None, 784], dtype=tf.float32)
reshape_images = tf.reshape(images, [-1, 28, 28, 1])

labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)

In [6]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

In [7]:
conv_layer1 = tf.nn.relu(conv(reshape_images, W_conv1) + b_conv1)
pool_layer1 = max_pool(conv_layer1)

conv_layer2 = tf.nn.relu(conv(pool_layer1, W_conv2) + b_conv2)
pool_layer2 = max_pool(conv_layer2)

pool2_flat = tf.reshape(pool_layer2, [-1, 7*7*64])

with tf.name_scope('fc_layer') as scope:
    fc_layer1 = tf.nn.relu(tf.matmul(pool2_flat, W_fc1) + b_fc1)

with tf.name_scope('fc_layer') as scope:
    keep_prob = tf.placeholder(tf.float32)
    drop_out = tf.nn.dropout(fc_layer1, keep_prob)
    logit = tf.matmul(drop_out, W_fc2) + b_fc2

In [8]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logit)
)
train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(logit, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.image('images', reshape_images)
tf.summary.scalar('xentropy', loss)
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()

BATCH_SIZE = 50
max_epoch = 5
total_batch = int(mnist.train.num_examples/BATCH_SIZE)
# with tf.Session() as sess:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
test_writer = tf.summary.FileWriter('./test', sess.graph)
for epoch in range(max_epoch):
    for i in range(total_batch):
        batch = mnist.train.next_batch(50, shuffle=True)
        sess.run(train_op, 
             feed_dict={images: batch[0], labels: batch[1], keep_prob: 0.5}
        )
        train_acc = sess.run(accuracy,
            feed_dict={images: batch[0], labels: batch[1], keep_prob: 1.0}
        )
        print('step %d, training accuracy %g' % (i, train_acc))
        if i % 100 == 0:
            test_acc, summary = sess.run([accuracy, merged], 
                 feed_dict={images: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}
            )
            print('test accuracy %g' % test_acc)
            test_writer.add_summary(summary, i)
test_acc, summary = sess.run([accuracy, merged], 
     feed_dict={images: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}
)
print('test accuracy %g' % test_acc)
test_writer.add_summary(summary, i)

step 0, training accuracy 0.12
test accuracy 0.1019
step 1, training accuracy 0.06
step 2, training accuracy 0.08
step 3, training accuracy 0.06
step 4, training accuracy 0.12
step 5, training accuracy 0.14
step 6, training accuracy 0.2
step 7, training accuracy 0.16
step 8, training accuracy 0.16
step 9, training accuracy 0.22
step 10, training accuracy 0.2
step 11, training accuracy 0.24
step 12, training accuracy 0.2
step 13, training accuracy 0.32
step 14, training accuracy 0.26
step 15, training accuracy 0.3
step 16, training accuracy 0.4
step 17, training accuracy 0.4
step 18, training accuracy 0.56
step 19, training accuracy 0.4
step 20, training accuracy 0.4
step 21, training accuracy 0.56
step 22, training accuracy 0.52
step 23, training accuracy 0.44
step 24, training accuracy 0.5
step 25, training accuracy 0.52
step 26, training accuracy 0.56
step 27, training accuracy 0.44
step 28, training accuracy 0.46
step 29, training accuracy 0.64
step 30, training accuracy 0.5
step 31

step 252, training accuracy 0.96
step 253, training accuracy 0.98
step 254, training accuracy 0.92
step 255, training accuracy 0.92
step 256, training accuracy 0.9
step 257, training accuracy 0.94
step 258, training accuracy 0.94
step 259, training accuracy 0.88
step 260, training accuracy 0.9
step 261, training accuracy 0.92
step 262, training accuracy 1
step 263, training accuracy 0.88
step 264, training accuracy 0.92
step 265, training accuracy 0.94
step 266, training accuracy 0.94
step 267, training accuracy 0.86
step 268, training accuracy 0.94
step 269, training accuracy 0.92
step 270, training accuracy 0.92
step 271, training accuracy 0.92
step 272, training accuracy 0.96
step 273, training accuracy 0.96
step 274, training accuracy 0.94
step 275, training accuracy 0.94
step 276, training accuracy 0.86
step 277, training accuracy 0.96
step 278, training accuracy 0.96
step 279, training accuracy 0.88
step 280, training accuracy 0.86
step 281, training accuracy 0.96
step 282, train

In [9]:
summary

b'\n\x0f\n\x08xentropy\x15~\x8d+>\n\x0f\n\x08accuracy\x158\xf8r?'

Tensorboard를 통해 중요한 요소들을 기록하는 코드를 추가했다.
가장 먼저 loss가 정상적으로 줄어드는지 확인하기 위해 추가하되, loss는 스칼라값이므로 tf.summary.scalar를 통해 기록한다. 이 때 앞에 있는 parameter는 기록할 name이다.
accuracy 또한 iteration마다 어떻게 증가하는지 확인하기 위해 기록한다.

tf.summary.merge_all 함수는 tf.summary를 통해 기록하기 위해 넣은 operation을 모두 병합하여 실행하는 함수다. tf.summary.FileWriter는 기록한 내용을 파일에 쓰는 함수이다. 원하는 폴더를 argument로 넘겨주면 된다. 우리는 test set에 대한 스칼라값만 기록할 것이므로 test 폴더에 기록한다.

sess.run을 통해 test set의 accuracy operation을 실행할 때, merged operation도 함께 실행한다.

파일에 기록한 summary를 TensorBoard로 확인하기 위해서는 해당 폴더에 접근하여,

~\test> tensorboard --logdir=.

위 명령어를 입력하면 된다.
그럼 웹브라우저에서 http://localhost:6006 를 통해 확인할 수 있다.